In [1]:
!pip install mrtparse
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.14.1-py3-none-any.whl size=267559 sha256=a0b2a0d20b676c3da896ccbb7f507dcb70c053e66b8e919c3db6001141b964db
  Stored in directory: /root/.cache/pip/wheels/13/01/ff/de2142e172dafcd475f118499db2140c66562e42fd500c460a
Successfully built neo4j


In [6]:
import datetime
import requests
import os
import json
import gzip
import shutil
from tqdm import tqdm
from mrtparse import Reader
from neo4j import GraphDatabase

In [4]:
def download_gz_file(url, filename):
    """
    Download a .gz file from the given URL and save it as filename.
    """
    response = requests.get(url, stream=True)
    # Check if the response was successful
    if response.status_code == 200:
        # Save the file to disk
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {filename}")
    else:
        # Print an error message if the download failed
        print(f"Failed to download. Status code: {response.status_code}")

In [15]:
now = datetime.datetime.now()
today = datetime.date.today()
# month = now.month
# year = now.year
# hour = now.hour
# base_url = "https://data.ris.ripe.net/rrc11/"
# url = base_url + str(year) + "." + str(month)
# part1 = now.strftime("%Y%m%d")
# part2 = now.strftime("%H00")
# url += "/updates." + part1 + "." + part2 + ".gz"

# Example Data, to comply to our bviews data

url = "https://data.ris.ripe.net/rrc03/2007.08/updates.20070801.1600.gz"
download_gz_file(url, f'{today}.gz')

Downloaded: 2023-11-27.gz


In [16]:
with gzip.open(f'{today}.gz', 'rb') as f_in:
  with open(f'{today}.txt', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

message_list = []
for entry in Reader(f'{today}.txt'):
    message_list.append(entry.data)


In [17]:
len(message_list)

5395

In [18]:
type_list = []
for bgp in message_list:
  if 'bgp_message' in bgp:
    if bgp['bgp_message']['type']:
      type_list.append(bgp['bgp_message']['type'])
type_list = [list(x.values())[0] for x in type_list]
set(type_list)

{'KEEPALIVE', 'UPDATE'}

In [22]:
test = message_list[10]
test

OrderedDict([('timestamp', {1185984013: '2007-08-01 16:00:13'}),
             ('type', {16: 'BGP4MP'}),
             ('subtype', {1: 'BGP4MP_MESSAGE'}),
             ('length', 82),
             ('peer_as', '29686'),
             ('local_as', '12654'),
             ('ifindex', 0),
             ('afi', {1: 'IPv4'}),
             ('peer_ip', '195.69.145.56'),
             ('local_ip', '195.69.144.69'),
             ('bgp_message',
              OrderedDict([('marker',
                            'ff ff ff ff ff ff ff ff ff ff ff ff ff ff ff ff'),
                           ('length', 66),
                           ('type', {2: 'UPDATE'}),
                           ('withdrawn_routes_length', 0),
                           ('withdrawn_routes', []),
                           ('path_attributes_length', 39),
                           ('path_attributes',
                            [OrderedDict([('flag', 64),
                                          ('type', {1: 'ORIGIN'}),
             

In [23]:
test['bgp_message']

OrderedDict([('marker', 'ff ff ff ff ff ff ff ff ff ff ff ff ff ff ff ff'),
             ('length', 66),
             ('type', {2: 'UPDATE'}),
             ('withdrawn_routes_length', 0),
             ('withdrawn_routes', []),
             ('path_attributes_length', 39),
             ('path_attributes',
              [OrderedDict([('flag', 64),
                            ('type', {1: 'ORIGIN'}),
                            ('length', 1),
                            ('value', {0: 'IGP'})]),
               OrderedDict([('flag', 64),
                            ('type', {2: 'AS_PATH'}),
                            ('length', 10),
                            ('value',
                             [OrderedDict([('type', {2: 'AS_SEQUENCE'}),
                                           ('length', 4),
                                           ('value',
                                            ['29686',
                                             '1299',
                                   

In [34]:
test['bgp_message']['nlri'][0]['prefix']

'66.194.138.0'

In [37]:
[str(x) for x in list(test['bgp_message'].get('path_attributes',[]))]

["OrderedDict([('flag', 64), ('type', {1: 'ORIGIN'}), ('length', 1), ('value', {0: 'IGP'})])",
 "OrderedDict([('flag', 64), ('type', {2: 'AS_PATH'}), ('length', 10), ('value', [OrderedDict([('type', {2: 'AS_SEQUENCE'}), ('length', 4), ('value', ['29686', '1299', '7018', '31957'])])])])",
 "OrderedDict([('flag', 64), ('type', {3: 'NEXT_HOP'}), ('length', 4), ('value', '195.69.145.56')])",
 "OrderedDict([('flag', 192), ('type', {8: 'COMMUNITY'}), ('length', 12), ('value', ['29686:600', '29686:601', '29686:801'])])"]

# Save into database

In [39]:
from neo4j import GraphDatabase

# Function to process each OrderedDict
def process_ordered_dict(tx, bgp_data,message_id):
    # Convert all keys to strings and check for 'bgp_message'
    bgp_data = {str(key): value for key, value in bgp_data.items()}
    if 'bgp_message' not in bgp_data:
        # Skip this row if 'bgp_message' is not present
        return

    local_as = bgp_data.get('local_as')
    peer_as = bgp_data.get('peer_as')
    timestamp = list(bgp_data['timestamp'].values())[0]

    bgp_message = bgp_data['bgp_message']
    message_type = list(bgp_message['type'].values())[0]
    message_length = bgp_message.get('length')

    # Create or merge Local AS and Peer AS nodes
    tx.run("MERGE (localAS:AS {id: $local_as})", local_as=local_as)
    tx.run("MERGE (peerAS:AS {id: $peer_as})", peer_as=peer_as)

    if message_type == 'UPDATE':
      withdrawn_routes_length = len(bgp_message.get('withdrawn_routes', []))
      nlri_length = len(bgp_message.get('nlri', []))
      is_withdrawal = withdrawn_routes_length > 0
      is_announcement = nlri_length > 0

      announcements_history = {}
      # Create relationship based on message type
      if is_withdrawal:
        withdrawn_list = [x['prefix'] for x in bgp_message['withdrawn_routes']]
        tx.run("""
              MATCH (localAS:AS {id: $local_as}), (peerAS:AS {id: $peer_as})
              CREATE (localAS)-[r:WITHDRAWS_ROUTE_TO {m_id: $message_id}]->(peerAS)
              SET r.timestamp = $timestamp, r.message_type = $message_type, r.message_length = $message_length, r.routes_length = $withdrawn_routes_length, r.withdrawn_list = $withdrawn_list
              """, local_as=local_as, peer_as=peer_as,timestamp=timestamp,
               message_type=message_type, message_length=message_length,
               withdrawn_routes_length=withdrawn_routes_length,message_id=message_id, withdrawn_list=withdrawn_list)
      if is_announcement:
        path_seq = list(bgp_message['path_attributes'][1]['value'][0]['value'])
        prefix = bgp_message['nlri'][0]['prefix']
        announcement_key = (local_as, peer_as, prefix)
        attributes = [str(x) for x in list(bgp_message.get('path_attributes',[]))]
        # Check for new, duplicate, or implicit withdrawal
        if announcement_key not in announcements_history:
            category = 'new_announcement'
        elif announcements_history[announcement_key] == attributes:
            category = 'duplicate_announcement'
        else:
            category = 'implicit_withdrawal'
        tx.run("""
              MATCH (localAS:AS {id: $local_as}), (peerAS:AS {id: $peer_as})
              CREATE (localAS)-[r:ANNOUNCES_ROUTE_TO {m_id: $message_id, category:$category}]->(peerAS)
              SET r.timestamp = $timestamp, r.message_type = $message_type, r.message_length = $message_length,
              r.routes_length = $nlri_length, r.path_seq = $path_seq
              """, local_as=local_as, peer_as=peer_as, timestamp=timestamp,
               message_type=message_type, message_length=message_length,
               nlri_length=nlri_length,message_id=message_id, path_seq=path_seq, category=category)
    elif message_type == 'OPEN':
      # Create relationship based on message type
      open_version = bgp_message.get('version')
      open_local_as = bgp_message.get('local_as')
      open_holdtime = bgp_message.get('holdtime')
      open_bgp_id = bgp_message.get('bgp_id')
      # Handle OPEN message
      tx.run("""
            MATCH (localAS:AS {id: $local_as}), (peerAS:AS {id: $peer_as})
            CREATE (localAS)-[r:OPENS_CONNECTION_TO {m_id: $message_id}]->(peerAS)
            SET r.timestamp = $timestamp, r.message_length = $message_length,
                r.version = $version, r.open_local_as = $open_local_as,
                r.holdtime = $holdtime, r.bgp_id = $bgp_id
            """, local_as=local_as, peer_as=peer_as, timestamp=timestamp,
                message_length=message_length, version=open_version,
                open_local_as=open_local_as, holdtime=open_holdtime, bgp_id=open_bgp_id,message_id=message_id)
    elif message_type == 'NOTIFICATION':
      # Handle NOTIFICATION message
      error_code = list(bgp_message['error_code'].values())[0]
      error_subcode = list(bgp_message.get('error_subcode'))[0]
      # Handle NOTIFICATION message
      tx.run("""
            MATCH (localAS:AS {id: $local_as}), (peerAS:AS {id: $peer_as})
            CREATE (localAS)-[r:SENDS_NOTIFICATION_TO {m_id: $message_id}]->(peerAS)
            SET r.timestamp = $timestamp, r.message_length = $message_length,
                r.error_code = $error_code, r.error_subcode = $error_subcode
            """, local_as=local_as, peer_as=peer_as, timestamp=timestamp,
                message_length=message_length, error_code=error_code,
                error_subcode=error_subcode,message_id=message_id)
    elif message_type == 'KEEPALIVE':
      # Handle KEEPALIVE message
      tx.run("""
            MATCH (localAS:AS {id: $local_as}), (peerAS:AS {id: $peer_as})
            CREATE (localAS)-[r:SENDS_KEEPALIVE_TO {m_id: $message_id}]->(peerAS)
            SET r.timestamp = $timestamp, r.message_length = $message_length
            """, local_as=local_as, peer_as=peer_as, timestamp=timestamp, message_length=message_length,message_id=message_id)
# Connect to Neo4j
driver = GraphDatabase.driver("neo4j+s://896675cf.databases.neo4j.io", auth=("neo4j", "vDR6onHDPNATBPsRKm4Crvk7EDSii3AuNTKxX_DZgdM"))

# Process each OrderedDict
with driver.session() as session:
  for i, bgp_dict in enumerate(tqdm(message_list, desc="Processing BGP Messages")):
        session.execute_write(process_ordered_dict, bgp_dict, message_id=i)
driver.close()


Processing BGP Messages:  10%|▉         | 519/5395 [07:05<1:09:39,  1.17it/s]ERROR:neo4j:Failed to read from defunct connection IPv4Address(('896675cf.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.69.128.95', 7687)))
ERROR:neo4j:Failed to read from defunct connection ResolvedIPv4Address(('34.69.128.95', 7687)) (ResolvedIPv4Address(('34.69.128.95', 7687)))
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
Processing BGP Messages: 100%|██████████| 5395/5395 [1:13:53<00:00,  1.22it/s]
